In [ ]:
import spacy
import numpy as np
from correlation import *
from spacy import displacy

nlp = spacy.load("en_core_web_sm")

In [30]:
from textutils import importer
import json

annual_report = importer.TextImporter("../texts/AnnualReport2017-2018.txt")
with open("../ndc_keywords/ndc_south_africa.json") as f:
    ndc_keywords = json.load(f)

climate_keywords = ndc_keywords['climate change']

In [24]:
token_correlator = TokenArrayCorrelator(climate_keywords, 0.4, "CLIMATE_TOKEN")
span_correlator = SpanCorrelator(climate_keywords, 0.4, "CLIMATE_SPAN")
generic_correlator = KeywordCorrelator(climate_keywords)


In [ ]:
print(climate_keywords)

# First, explore the generic correlator functionality

In [23]:
sentence_to_correlate = "We need to adapt our project to be more resillient to geographical circumstances"
unrelated_sentence = "The next time the leaders will meet in paris"

print(generic_correlator([sentence_to_correlate, unrelated_sentence]))

[0.4462148  0.05690926 0.36428687]


We can see that the sentence which uses some of the NDC keywords has a much higher embedding score than the unrelated sentence

# Now, lets see how we can integrate the output of our embedder into Spacy, an NLP library

In [20]:
# Load our text into a spacy document
doc = nlp(annual_report.text)

In [32]:
doc.ents = []
from n_gram_correlation import NGramCorrelateSpacy

n_gram_cor = NGramCorrelateSpacy(climate_keywords, 0.7, "CLIMATE_N")

n_gram_cor.correlate_spans(doc, 7)

KeyboardInterrupt: 

In [26]:
print(doc.ents)

(Strengthening Cooperative Government
e. Deepening International Engagement
f. Institutional Capacity and Support
5., Programme 2: Administration
iv., 
Overall Performance
, 
Performance
Indicator
Annual
Performance
target
Performance, 
Performance
Indicator
Annual
Performance
target
Performance, 
Performance
Indicator
Annual
Performance
target
Performance, 
Performance
Indicator
Annual
Performance
target
Performance, 
+4,74, 
Performance
Indicator
Annual
Performance
target
Performance, 
Performance
Indicator
Annual
Performance
target
Performance, 
Performance
Indicator
Annual
Performance
target
Performance, 
Performance
Indicator
Annual
Performance
target
Performance, 
Performance
Indicator
Annual
Performance
target
Performance, 
Performance
Indicator
Annual
Performance
target
Performance, 
Performance
Indicator
Annual
Performance
target
Performance, Adjustments, The FMPPLA defines unauthorised expenditure as: (a) overspending of Parliament's approved budget or a main division
within 